<div style=" border-bottom: 8px solid #00B2FF; overflow: hidden; border-radius: 10px; height: 60px; width: 100%; display: flex;">
  <div style="height: 100%; width: 100%; background-color: #3800BB; float: left; text-align: center; display: flex; justify-content: left; align-items: center; font-size: 40px; ">
    <b><span style="color: #FFFFFF; padding: 20px 20px;">LightGBM</span></b>
  </div>
</div>

<div class="alert" style="background-color: #FEDAD5; border-left: 8px solid #B12111; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">
  <h5 style="font-size: 16px; margin-bottom: 10px;">
    <strong> Contents </strong>
  </h5>
<hr>
  <p><font size="3" face="Arial" font-size="large">
  <ul type="square">
      
<li> LightGBM Classifier and its Parameters </li>
<li> Wrapping Data in a Dataset Class </li>
<li> Two Types of Models in LightGBM </li>
<li> Key Parameters </li>
<li> Tuning Optimal Hyperparameters </li>
<li> Built-in Visualization and Feature Importance </li>
<li> Custom Loss & Eval Functions </li>
<li> Conclusions and Summary </li>

  
  </ul>
  </font></p>

</div>

<div class="alert alert-warning">

### **Why do we need [LightGBM](https://lightgbm.readthedocs.io/en/v3.3.2/)?**
</div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

- **`LightGBM`** has a similar interface to **`CatBoost`**
- It comes with its own quirks and hacks when it comes to tuning
- Naturally, it has unique advantages in functionality
- We'll use `LightGBM` as a deeper dive into boosting configuration
- Surprisingly, sometimes a specific boosting library outperforms the rest
- We'll go through some fancy features and leave the tricky nuances for practice

</div>

<div class="alert" style="background-color:  #E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

`LGBM (Light Gradient Boosted Machine)` was introduced by Ke G. et al. in their 2017 paper [“LightGBM: A Highly Efficient Gradient Boosting Decision Tree”](https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree).

The implementation is based on two key ideas: `GOSS` and `EFB`.<br>
The framework is developed and maintained by Microsoft.

</div>